In [1]:
import spacy
# python3 -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
# %pip install datasets
import numpy as np
from gensim.models import KeyedVectors
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import datasets
nltk.download('stopwords')
from datasets import load_dataset

/home/vyakhya/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vyakhya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = load_dataset("paws", "labeled_final")
# Access train, dev, and test splits
train_data = dataset["train"]
dev_data = dataset["validation"]
test_data = dataset["test"]

In [3]:
def preprocess_sentence(sentence):
    # Tokenize the sentence using SpaCy
    doc = nlp(sentence)
    
    # Lemmatize tokens and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    
    return tokens



In [ ]:
i = 0
len_tr = len(train_data)
# Preprocess the sentences in the train split
train_data_preprocessed = []
for example in train_data:
    i+=1
    if(i>len_tr//2):
        break
    example["preprocessed_sentence1"] = preprocess_sentence(example["sentence1"])
    example["preprocessed_sentence2"] = preprocess_sentence(example["sentence2"])
    train_data_preprocessed.append(example)

# Preprocess the sentences in the dev split
dev_data_preprocessed = []
for example in dev_data:
    example["preprocessed_sentence1"] = preprocess_sentence(example["sentence1"])
    example["preprocessed_sentence2"] = preprocess_sentence(example["sentence2"])
    dev_data_preprocessed.append(example)


In [4]:

# Preprocess the sentences in the test split
test_data_preprocessed = []
for example in test_data:
    example["preprocessed_sentence1"] = preprocess_sentence(example["sentence1"])
    example["preprocessed_sentence2"] = preprocess_sentence(example["sentence2"])
    test_data_preprocessed.append(example)


In [5]:
import gensim

def tagged_document(list_of_words, idx):
    return gensim.models.doc2vec.TaggedDocument(list_of_words, [idx])

In [ ]:
# Create TaggedDocument objects for each pair of sentences in the train split
train_tagged_data = []
for idx, example in enumerate(train_data_preprocessed):
    sentence1 = example["preprocessed_sentence1"]
    sentence2 = example["preprocessed_sentence2"]
    tagged_doc1 = tagged_document(sentence1, idx)
    tagged_doc2 = tagged_document(sentence2, idx)
    train_tagged_data.extend([tagged_doc1, tagged_doc2])

# Create TaggedDocument objects for each pair of sentences in the dev split
dev_tagged_data = []
for idx, example in enumerate(dev_data_preprocessed):
    sentence1 = example["preprocessed_sentence1"]
    sentence2 = example["preprocessed_sentence2"]
    tagged_doc1 = tagged_document(sentence1, idx)
    tagged_doc2 = tagged_document(sentence2, idx)
    dev_tagged_data.extend([tagged_doc1, tagged_doc2])

In [6]:
# Create TaggedDocument objects for each pair of sentences in the test split
test_tagged_data = []
for idx, example in enumerate(test_data_preprocessed):
    sentence1 = example["preprocessed_sentence1"]
    sentence2 = example["preprocessed_sentence2"]
    tagged_doc1 = tagged_document(sentence1, idx)
    tagged_doc2 = tagged_document(sentence2, idx)
    test_tagged_data.extend([tagged_doc1, tagged_doc2])

In [7]:
vector_size = 300  
epochs = 30
model = gensim.models.doc2vec.Doc2Vec(vector_size=vector_size, epochs=epochs)
model.build_vocab(train_tagged_data)
model.train(train_tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("models/doc2vecsent")

In [7]:
model_path = "models/doc2vecsent"
loaded_model = gensim.models.doc2vec.Doc2Vec.load(model_path)
# Function to compute similarity between two sentences


In [8]:
# Convert test_data to pandas DataFrame
import pandas as pd
test_df = pd.DataFrame(test_data_preprocessed)

# # Preprocess the sentences in the test set
# test_df['preprocessed_sentence1'] = test_df['sentence1'].apply(preprocess_sentence)
# test_df['preprocessed_sentence2'] = test_df['sentence2'].apply(preprocess_sentence)


In [9]:

from sklearn.metrics.pairwise import cosine_similarity
model=loaded_model
# Function to compute similarity between two sentences
def compute_similarity(sentence1, sentence2):
    vector1 = model.infer_vector(sentence1)
    vector2 = model.infer_vector(sentence2)
    similarity_score = cosine_similarity([vector1], [vector2])[0][0]
    return similarity_score


# Calculate similarity scores for each pair of sentences in the test set
test_df['predicted_similarity_score'] = test_df.apply(lambda x: compute_similarity(x['preprocessed_sentence1'], x['preprocessed_sentence2']), axis=1)

# Classify based on similarity score
test_df['predicted_label'] = test_df['predicted_similarity_score'].apply(lambda x: 1 if x >= 0.5 else 0)

# Report Results
accuracy = (test_df['predicted_label'] == test_df['label']).mean()
print(f"Accuracy on the test set: {accuracy}")


Accuracy on the test set: 0.44675


In [10]:
# Load pre-trained word embeddings (Word2Vec)
word2vec_model_path_pretrained = 'models/GoogleNews-vectors-negative300.bin.gz'
word2vec_model_pretrained = KeyedVectors.load_word2vec_format(word2vec_model_path_pretrained, binary=True)

In [11]:
def get_sentence_embedding(sentence):
    embeddings = [word2vec_model_pretrained[word] for word in sentence if word in word2vec_model_pretrained]  # Get word embeddings
    if embeddings:
        return np.mean(embeddings, axis=0)  # Average the word embeddings to get the sentence embedding
    else:
        return np.zeros(word2vec_model_pretrained.vector_size)  # Return zeros if no embeddings found

def compute_similarity(sentence1, sentence2):
    vector1 = get_sentence_embedding(sentence1)
    vector2 = get_sentence_embedding(sentence2)
    similarity_score = cosine_similarity([vector1], [vector2])[0][0]
    return similarity_score

# Function to predict sentence similarity
def predict_similarity(sentence1, sentence2, threshold=0.5):
    # Get embeddings for each sentence
    embedding1 = get_sentence_embedding(sentence1)
    embedding2 = get_sentence_embedding(sentence2)
    
    # Calculate cosine similarity between the embeddings
    similarity_score = cosine_similarity([embedding1], [embedding2])[0][0]
    
    # Classify as similar or not based on the threshold
    if similarity_score >= threshold:
        return 1
    else:
        return 0
test_df['predicted_similarity_score'] = test_df.apply(lambda x: compute_similarity(x['preprocessed_sentence1'], x['preprocessed_sentence2']), axis=1)

# Classify based on similarity score
test_df['predicted_label'] = test_df['predicted_similarity_score'].apply(lambda x: 1 if x >= 0.5 else 0)

# Report Results
accuracy = (test_df['predicted_label'] == test_df['label']).mean()
print(f"Accuracy on the test set: {accuracy}")



Accuracy on the test set: 0.442125
